Lab 2: Data Preparation

## A. Import Python Modules and Dataset

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/mandricic/CS5062/master/bank.csv')
df.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed
1995,20.0,student,single,NaN,no,2785,no,no,cellular,16,sep,327,2,-1,0,NaN,yes
1996,28.0,admin.,single,secondary,no,127,no,no,cellular,16,sep,1334,2,-1,0,NaN,yes
1997,81.0,retired,married,primary,no,1154,no,no,telephone,17,sep,231,1,-1,0,NaN,yes
1998,46.0,services,married,primary,no,4343,yes,no,NaN,20,sep,185,1,-1,0,NaN,yes
1999,40.0,entrepreneur,married,secondary,no,6403,no,no,cellular,22,sep,208,2,-1,0,NaN,yes


Check for missing values
we can Use df.isna().sum() or df.isnull().sum()

In [3]:
df.isnull().sum()

age            12
job            10
marital         0
education     104
default         0
balance         0
housing         0
loan            0
contact       191
day             0
month           0
duration        0
campaign        0
pdays           0
previous        0
poutcome      454
subscribed      0
dtype: int64

Save Original data

In [0]:
df_original = df

### Encode Ordinal Categorical Attributes 

for Month we use Categorical Attribute, I will replace that with Numerical Attributes

In [5]:
month_mapper = {
    "jan": 1,
    "feb": 2,
    "mar": 3,
    "apr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "aug": 8,
    "sep":9,
    "oct":10,
    "nov":11,
    "dec":12
}

df['month'] = df['month'].replace(month_mapper)

df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed
0,32.0,technician,single,tertiary,no,392,yes,no,cellular,1,4,957,2,131,2,failure,no
1,39.0,technician,divorced,secondary,no,688,yes,yes,cellular,1,4,233,2,133,1,failure,no
2,59.0,retired,married,secondary,no,1035,yes,yes,cellular,1,4,126,2,239,1,failure,no
3,47.0,blue-collar,married,secondary,no,398,yes,yes,cellular,1,4,274,1,238,2,failure,no
4,54.0,retired,married,secondary,no,1004,yes,no,cellular,1,4,479,1,307,1,failure,no


One-Hot Encode Education Categorical Attribute

In [6]:
df = pd.get_dummies(df, columns=['education'], 
                        dummy_na=True, 
                        drop_first=True)
df.tail()

,age,job,marital,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed,education_secondary,education_tertiary,education_nan
1995,20.0,student,single,no,2785,no,no,cellular,16,9,327,2,-1,0,NaN,yes,0,0,1
1996,28.0,admin.,single,no,127,no,no,cellular,16,9,1334,2,-1,0,NaN,yes,1,0,0
1997,81.0,retired,married,no,1154,no,no,telephone,17,9,231,1,-1,0,NaN,yes,0,0,0
1998,46.0,services,married,no,4343,yes,no,NaN,20,9,185,1,-1,0,NaN,yes,0,0,0
1999,40.0,entrepreneur,married,no,6403,no,no,cellular,22,9,208,2,-1,0,NaN,yes,1,0,0


In [7]:
df.isnull().sum()

age                     12
job                     10
marital                  0
default                  0
balance                  0
housing                  0
loan                     0
contact                191
day                      0
month                    0
duration                 0
campaign                 0
pdays                    0
previous                 0
poutcome               454
subscribed               0
education_secondary      0
education_tertiary       0
education_nan            0
dtype: int64

Impute Missing Values

In [8]:
df['poutcome'].fillna('novalue', inplace=True)
df.tail()

,age,job,marital,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed,education_secondary,education_tertiary,education_nan
1995,20.0,student,single,no,2785,no,no,cellular,16,9,327,2,-1,0,novalue,yes,0,0,1
1996,28.0,admin.,single,no,127,no,no,cellular,16,9,1334,2,-1,0,novalue,yes,1,0,0
1997,81.0,retired,married,no,1154,no,no,telephone,17,9,231,1,-1,0,novalue,yes,0,0,0
1998,46.0,services,married,no,4343,yes,no,NaN,20,9,185,1,-1,0,novalue,yes,0,0,0
1999,40.0,entrepreneur,married,no,6403,no,no,cellular,22,9,208,2,-1,0,novalue,yes,1,0,0
